# Statistical Machine Learning Project1 Source Code

**Group 68**

Team Members: Rongxiao Liu; Xudong Ma; Dong Jia

### Load Data

In [0]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import csv

In [0]:
# load train set
data = pd.read_csv('./drive/My Drive/SML_Project1/whodunnit/train_tweets.txt',names=["userid","text"],sep='\t',quoting=csv.QUOTE_NONE)
# data = data[data["userid"] < 1000]
classes = np.array(list(set(data["userid"])))
print(data.head(5))
print("labelled tweets size: {}".format(data.shape[0]))

   userid                                               text
0    8746     @handle Let's try and catch up live next week!
1    8746  Going to watch Grey's on the big screen - Thur...
2    8746  @handle My pleasure Patrick....hope you are well!
3    8746  @handle Hi there! Been traveling a lot and lot...
4    8746  RT @handle Looking to Drink Clean & Go Green? ...
labelled tweets size: 328932


In [0]:
data2predict = pd.read_csv('./drive/My Drive/SML_Project1/whodunnit/test_tweets_unlabeled.txt',names=["text"],sep='\t',quoting=csv.QUOTE_NONE)
print(data2predict.head(5))
print("unlabelled tweets size: {}".format(data2predict.shape[0]))

                                                text
0  Some people say that rappers don’t have feelin...
1  Do you know how to tweet on a Blackberry 8830?...
2        "Yoga is the cessation of mind." -Patanjali
3  @handle Well, with my millions of dollars, a f...
4  Cambria hotels free guide http://hotels.izigot...
unlabelled tweets size: 35437


### Feature Extraction

In [0]:
import re
import socket
import pandas as pd
import numpy as np
import string
import csv
from nltk.tokenize import TweetTokenizer
import preprocessor as p
def char_num(x): #字符数
	return len(x.text)
def retweet(x):
	n = re.findall(r"\bRT\b",x.text)
	return 1 if len(n) != 0 else 0
def url(x):
	#m = re.findall(r'(?<!//)(www.)[\w./]*',x.text)]
	m = re.findall(r"(?<!//)\bwww.[\w.]*[\S]*",x.text)
	n = re.findall(r'(?:http|https)://(?:[\w_-]+(?:(?:\.[\w_-]+)+))',x.text)
	return len(n) + len(m)
def hashtage(x):
	n = re.findall(r'#(\w*[0-9a-zA-Z]+\w*[0-9a-zA-Z])',x.text)
	return 1 if len(n) != 0 else 0
def if_mention(x):
	n = re.findall(r'@\bhandle\b',x.text)
	return len(n)
def spec_char(x):
	n = re.findall(r"((?=[\x21-\x7e]+)[^A-Za-z0-9])",x.text)
	return len(n)
def punc_num(x): #标点数
	i = 0
	for char in x.text:
		if char in punc_list:
			i += 1
	return i
def spec_num(x): #特殊字符，不包括@,不包括网站
	i = 0
	temp = ' '.join([token if not token.startswith("http") else 'http' for token in x.text.split()])
	for char in temp:
		if char in spec_list:
			i += 1
	return i
def continue_punc(x):
	n = re.findall(r'[A-Za-z]+[!?]{2,}',x.text)
	return len(n)
def word_num(x):
	i = 1
	for token in x.text.split():
		if token.isalpha():
			i += 1
	return i
def sent_num(x):
	i = x.text.split(".")
	return len(i)
def smiley_num(x): #笑脸的数量 时间略长500s
	i = 0
	for token in p.tokenize(x.text).split():
		if token in emoji_smiley:
			i += 1
	return i
def initcap(): #首字母大写
	i = 0
	for token in x.text.split():
		if token.istitle():
			i += 1
	return i
def allcap(x): #整个单词大写
	i = 0
	for token in x.text.split():
		if token.isupper():
			i += 1
	return i
def continue_char(x):
	n = re.findall(r"(\w)\1{2,}",x.text)
	return len(n)

In [0]:
# text clean function
import re
from nltk.tokenize import TweetTokenizer
def text_clean(text):
    text = text.lower()
    extend_list = []
    if len([c for c in text if c.isalpha()]) < len(text)/3:
        extend_list.append('$NONEN')
    if re.findall(r'(.)\1{2,}', text):
        for _ in range(len(re.findall(r'(.)\1{4,}', text))):
            extend_list.append('$REPEAT')
            
#     if re.findall(r'([a-zA-Z])\1{2,}', text):
#         for _ in range(len(re.findall(r'([a-zA-Z])\1{2,}', text))):
#             extend_list.append('$REPEAT')
#         extend_list.append('AAA')
#         text = re.sub(r'([a-zA-Z])\1{2,}', r'\1', text)

#     if re.findall(r'([\?!])\1{2,}', text):
#         for _ in range(len(re.findall(r'([\?!])\1{2,}', text))):
#             extend_list.append('$REPEAT')
#         extend_list.append('III')
#         text = re.sub(r'([\?!])\1{2,}', r' III ', text)

#     if re.findall(r'([\.])\1{2,}', text):
#         for _ in range(len(re.findall(r'([\.])\1{2,}', text))):
#             extend_list.append('$REPEAT')
#         extend_list.append('DDD')
#         text = re.sub(r'([\.])\1{2,}', r' DDD ', text)

    text_token = tweet_tokenizer.tokenize(text)
    for i, token in enumerate(text_token):
        if token.startswith("http://") or token.startswith("https://"):
            text_token[i] = token.split('/')[2]
#         if token.isdigit():
#             extend_list.append('$NUMBER')
    text_token.extend(extend_list)
    text = ' '.join(text_token)
    return text

In [0]:
# clean the data
tweet_tokenizer = TweetTokenizer()
data["processed_text"] = data["text"].apply(text_clean)  # clean train set
data2predict["processed_text"] = data2predict["text"].apply(text_clean)  # clean predict set
print(data["processed_text"].head(5))
data.to_csv('./drive/My Drive/SML_Project1/whodunnit/datatext.csv',index=False)

0      @handle let's try and catch up live next week !
1    going to watch grey's on the big screen - thur...
2    @handle my pleasure patrick ... hope you are w...
3    @handle hi there ! been traveling a lot and lo...
4    rt @handle looking to drink clean & go green ?...
Name: processed_text, dtype: object


In [0]:
# split into train and test data
from sklearn.model_selection import train_test_split
X_train_text, X_test_text, y_train, y_test = train_test_split(data["processed_text"], data["userid"],test_size=0.1, random_state=1234)

print("splited train set size: {}".format(X_train_text.shape[0]))
print("splited test set size: {}".format(X_test_text.shape[0]))

splited train set size: 296038
splited test set size: 32894


In [0]:
## 全拼起来
dataPlusPredict = X_train_text.append(X_test_text).append(data2predict["processed_text"])
dataPlusPredict.shape

(364369,)

In [0]:
## 拼起来之后整体取tfidf，再分开三份，限定词汇为train的词汇
# 提取tfidf特征_训练集
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=40000, analyzer='char',ngram_range=(5,5), binary=True,lowercase=False).fit(X_train_text)
vocab = tfidf_vectorizer.vocabulary_

tfidf_vectorizer1 = TfidfVectorizer(max_features=40000,analyzer='char',ngram_range=(5,5), binary=True,lowercase=False, vocabulary=vocab).fit(dataPlusPredict)

# tfidf_vectorizer1 = TfidfVectorizer(analyzer='char',ngram_range=(4,4),lowercase=False).fit(X_train_text)
X_all = tfidf_vectorizer1.transform(dataPlusPredict)
X_train = X_all[:X_train_text.shape[0]]
X_test = X_all[X_train_text.shape[0]:X_train_text.shape[0]+X_test_text.shape[0]]
X_predict = X_all[X_train_text.shape[0]+X_test_text.shape[0]:]

print("train set size: {}, feature: {}".format(X_train.shape[0], X_train.shape[1]))
print("test set size: {}, feature: {}".format(X_test.shape[0], X_test.shape[1]))
print("predict set size: {}, feature: {}".format(X_predict.shape[0], X_predict.shape[1]))

# print(vocab)
# print(tfidf_vectorizer.vocabulary_)
# print(tfidf_vectorizer2.vocabulary_)
# print(tfidf_vectorizer3.vocabulary_)

print(len(vocab),vocab)

train set size: 296038, feature: 40000
test set size: 32894, feature: 40000
predict set size: 35437, feature: 40000
40000 {'rt @h': 32362, 't @ha': 34786, ' @han': 1471, '@hand': 7446, 'handl': 18325, 'andle': 8943, 'ndle ': 25595, 'dle l': 12755, 'le la': 22585, 'e lau': 13677, ' laug': 3528, 'laugh': 22363, 'aughi': 10017, 'ughin': 37233, 'ghing': 17779, 'hing ': 18882, 'ing h': 20542, 'ng he': 25989, 'g hel': 17440, ' help': 3037, 'helps': 18690, 'elps ': 15128, 'ps ar': 30397, 's art': 32893, 'arter': 9453, 'eries': 15840, 'ries ': 31785, 'ies a': 19852, 'es an': 16010, 's and': 32884, ' and ': 1721, 'and b': 8910, 'nd bo': 25443, 'd boo': 11946, ' boos': 2045, 'boost': 10652, 'osts ': 29144, 'sts b': 34532, 's blo': 32928, ' bloo': 2026, 'blood': 10618, 'lood ': 23250, 'ood f': 28472, 'd flo': 12022, ' flow': 2794, 'flow ': 17090, 'low a': 23306, 'y.com': 39761, '.com ': 6944, 'com i': 11554, 'om i ': 27951, ' i gu': 3152, 'i gue': 19336, ' gues': 2963, 'guess': 18047, 'uess ': 37

In [0]:
# extract tfidf for train set
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer1 = TfidfVectorizer(analyzer='char',ngram_range=(4,4),binary=True,lowercase=False).fit(X_train_text)
X_train = tfidf_vectorizer1.transform(X_train_text)
vocab = tfidf_vectorizer1.vocabulary_  # get vocab in train set

# extract tfidf for test set
tfidf_vectorizer2 = TfidfVectorizer(analyzer='char',ngram_range=(4,4),binary=True,lowercase=False,vocabulary=vocab)
X_test = tfidf_vectorizer2.fit_transform(X_test_text)

# extract tfidf for predict set
tfidf_vectorizer3 = TfidfVectorizer(analyzer='char',ngram_range=(4,4),binary=True,lowercase=False,vocabulary=vocab)
X_predict = tfidf_vectorizer3.fit_transform(data2predict["processed_text"])

print("train set size: {}, feature: {}".format(X_train.shape[0], X_train.shape[1]))
print("test set size: {}, feature: {}".format(X_test.shape[0], X_test.shape[1]))
print("predict set size: {}, feature: {}".format(X_predict.shape[0], X_predict.shape[1]))

# print(vocab)
# print(tfidf_vectorizer.vocabulary_)
# print(tfidf_vectorizer2.vocabulary_)
# print(tfidf_vectorizer3.vocabulary_)

print(len(vocab),vocab)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Model Training
### SVM


In [0]:
# train model

from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(X_train, y_train)

# from sklearn.model_selection import GridSearchCV

# parameters = [
#     {
#         'C': [0.01, 0.1, 0.5, 0.8, 1, 2, 3, 5, 10, 20]
# ]
# clf = GridSearchCV(model, parameters, cv=5, n_jobs=8)
# clf.fit(X_train, y_train)
# print(clf.best_params_)
# model = clf.best_estimator_

In [0]:
# save model to local
from sklearn.externals import joblib
joblib.dump(model,'./drive/My Drive/SML_Project1/LinearSVC.model')
# model = joblib.load("./drive/My Drive/SML_Project1/LinearSVC.model")  # load model

In [0]:
# accuracy
from sklearn.metrics import accuracy_score
# predictions = model.predict(X_train)
# print("train accuracy: {}".format(accuracy_score(y_train, predictions)))
predictions = model.predict(X_test)
print("test accuracy: {}".format(accuracy_score(y_test, predictions)))

In [0]:
# predict unlabelled data
predictions = model.predict(X_predict)
print(predictions)

In [0]:
# save csv as format {"Id"  "Predicted"}
pred = pd.DataFrame({"Id": data2predict.index+1,"Predicted":predictions})
pred.to_csv("./drive/My Drive/SML_Project1/predictions.csv",index=False)

In [0]:
from sklearn.neural_network import MLPClassifier


mlp = MLPClassifier(solver='sgd', activation='relu',alpha=1e-4,hidden_layer_sizes=(50,50), random_state=1,max_iter=10,verbose=10,learning_rate_init=.1)
# mlp.fit(x_train, y_train) 

for t in range(2):
    for i in range(50):
        #第一次调用需要加classes
        left = i * 10000
        if x_train.shape[0] <= (i+1) * 10000:
            right = (i+1) * 10000
            mlp.partial_fit(x_train[left:right],y_train[left:right],classes)
        else:
            mlp.partial_fit(x_train[left:],y_train[left:],classes)
            break
        

print(mlp.score(x_test,y_test))

# pytorch network

**loading data**

In [0]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import CountVectorizer

data = pd.read_csv('./drive/My Drive/SML_Project1/whodunnit/train_tweets.txt',names=['userid','tweet'],sep='\t',quoting=csv.QUOTE_NONE)
# print(len(data['tweet']))
# print(data['tweet'][0])
count_vectorizer = CountVectorizer()
x_data = count_vectorizer.fit_transform(data['tweet']) #bag of word matrix
# x_data = x_data.tocoo()
# print('x type: {}'.format(type(x_data)))
y_data_raw = np.array(data['userid'])


#labelling authorid as numbers from 0
num_classes = len(set(y_data_raw))
labels = np.arange(0,num_classes)


label_dict = dict(zip(list(set(y_data_raw)),labels))
y_data = []
for data in y_data_raw:
  y_data.append(label_dict[data])
y_data = np.array(y_data)


#split train and test data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data, test_size=0.1, random_state=1234)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print('x type: {}'.format(type(x_train)))
print('y type: {}'.format(type(y_train)))
print('num_classes {}'.format(num_classes))

**csr2tensor**

In [0]:
import torch
from scipy import sparse
def csr2sparseTensor(x_csr):
  x_coo = x_csr.tocoo()
  data_shape = x_coo.shape
  data_value = x_coo.data
  data_index = np.vstack((x_coo.row,x_coo.col)) #(row[i],col[i])= data[i]
  tensor_index = torch.LongTensor(data_index)
  tensor_value = torch.FloatTensor(data_value)
  #create a tensor from a saving space format
  x_tensor = torch.sparse.FloatTensor(tensor_index,tensor_value,torch.Size(data_shape)) #.to_dense() can give the full matrix
  return x_tensor

**train and test (GPU)**

In [0]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import TensorDataset,DataLoader
from datetime import datetime
EPOCHES = 30
HIDDEN_UNITS = 1000
LR = 0.01
BATCH_SIZE = 1000
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class MY_CLASSIFIER(torch.nn.Module):
  def __init__(self,n_feature,n_hidden,n_output):
    super(MY_CLASSIFIER,self).__init__()
    self.h1 = torch.nn.Linear(n_feature,n_hidden)
    self.output = torch.nn.Linear(n_hidden,n_output)
#     print(self.h1)
#     print(self.output)
    
  def forward(self,x):
    x = F.relu(self.h1(x))
    x = self.output(x)
    return x
  

def train(x_train,y_train,num_classes):  
  #batch training
  #TensorDataset(x,y):x cannot be csr matrix
#   train_dataset = TensorDataset(x_train, y_train)
  
#   loader = DataLoader(
#       dataset=train_dataset,
#       batch_size=BATCH_SIZE,
#       num_workers=0
#   )
  start_train = datetime.now()
  num_features = x_train.shape[1]
  my_net = MY_CLASSIFIER(num_features,HIDDEN_UNITS,num_classes).to(device)
  total_paras = sum(p.numel() for p in my_net.parameters())
  total_paras_trainable = sum(p.numel() for p in my_net.parameters() if p.requires_grad)
  print('total_paras:{}\ntotal_paras_trainable:{}'.format(total_paras,total_paras_trainable))
  #print(my_net)
   
  #optimizer = torch.optim.Adam(my_net.parameters(),lr = LR, betas = (0.9,0.99))
  optimizer = torch.optim.SGD(my_net.parameters(),lr = LR)
  loss_func = torch.nn.CrossEntropyLoss()
  
  train_data_size = len(y_train)
  for epoch in range(EPOCHES):
    print('epoch {} has started'.format(epoch))
    for idx in range(0,train_data_size,BATCH_SIZE):
      if idx % 100000 == 0:
        time_cost = datetime.now() - start_train
        print("epoch: {} and idx: {} ; time: {}".format(epoch,idx,time_cost))
      x_batch = x_train[idx:idx+BATCH_SIZE]
      y_batch = y_train[idx:idx+BATCH_SIZE]
      
      #convert each batch of data to tensor and passing it to gpu step by step
      x_train_batch = Variable(csr2sparseTensor(x_batch),requires_grad=True).to(device)
      #print(x_train_batch.shape)
      y_train_batch = torch.LongTensor(y_batch).to(device) #32 bits
      
      #can be sparse on gpu ,but when feed to model have to be dense
      output = my_net(x_train_batch.to_dense())
      loss = loss_func(output,y_train_batch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  torch.save(my_net.state_dict(),'my_net.pkl')
  

def predict(x_test,num_classes):
  
  my_net = MY_CLASSIFIER(x_test.shape[1],HIDDEN_UNITS,num_classes).to(device)
  my_net.load_state_dict(torch.load('my_net.pkl'))
  
  #predict in small pieces
  test_data_size = x_test.shape[0]
  predictions = np.array([])
  for idx in range(0,test_data_size,BATCH_SIZE):
    x_batch = x_test[idx:idx+BATCH_SIZE]
    x_test_batch = csr2sparseTensor(x_batch).to(device)
    batch_predictions = my_net(x_test_batch.to_dense())
    batch_classifications = torch.max(F.softmax(batch_predictions),1)[1].cpu()
    predictions = np.hstack((predictions,batch_classifications.numpy()))
#   print(predictions.shape)
#   print(y_test.shape)
  return predictions

def accuracy_compution(y_test,predictions):
  correct_num = 0
  for i,y in enumerate(y_test):
    if predictions[i] == y:
      correct_num += 1
  return correct_num/len(y_test)
    
startTime = datetime.now()
train(x_train,y_train,num_classes)
predictions = predict(x_test,num_classes)
# print(predictions.shape)
accuracy = accuracy_compution(y_test,predictions)
print("accuracy: {}".format(accuracy))
endTime = datetime.now()
print('time: {}'.format(endTime-startTime))